In [3]:
import numpy as np
import pandas as pd
from util import save_result,preprocess_time

from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

In [4]:
test = pd.read_csv("../data/test.csv")
train = pd.read_csv("../data/train.csv")
test = preprocess_time(test)
train = preprocess_time(train)

In [3]:
#按 vender_id 取平均值
vendorids = train.vendor_id.unique()
avg_value_by_verndorid = {}
for i in vendorids:
    avg_value_by_verndorid[i] = train[train.vendor_id == i].trip_duration.mean()

print avg_value_by_verndorid
test["trip_duration"] = 0
for v_id in vendorids:
    test.loc[test.vendor_id == v_id,"trip_duration"] = avg_value_by_verndorid[v_id]
save_result(test[["id","trip_duration"]],"../result/vender_id_avg.csv")

{1: 845.43824354086871, 2: 1058.6431638006823}


In [7]:
# 按 时间 hour 取平均
avg_value_by_hour = {}
hours = train.hour.unique()
for h in hours:
    avg_value_by_hour[h] = train[train.hour==h].trip_duration.mean()
print avg_value_by_hour
for h in hours:
    test.loc[test.hour==h,"trip_duration"]=avg_value_by_hour[h]
save_result(test[["id","trip_duration"]],"../result/hour_avg.csv")

{0: 936.65730168269226, 1: 903.08711207902309, 2: 890.07693407693409, 3: 890.07087820052641, 4: 921.976443768997, 5: 822.29896013864823, 6: 797.43494345524539, 7: 831.75834532374097, 8: 924.5591994392496, 9: 933.52891535994559, 10: 933.36760548313646, 11: 966.43028214264848, 12: 993.53307918133373, 13: 1032.2463587648483, 14: 1075.7892639853551, 15: 1118.8325326203505, 16: 1080.1462845769906, 17: 1030.5816194448439, 18: 981.58323399558503, 19: 894.35667936395441, 20: 879.46154486630508, 21: 890.52122112015206, 22: 1023.4559210853253, 23: 925.09140932865228}


In [34]:
print train.columns

Index([u'id', u'vendor_id', u'pickup_datetime', u'dropoff_datetime',
       u'passenger_count', u'pickup_longitude', u'pickup_latitude',
       u'dropoff_longitude', u'dropoff_latitude', u'store_and_fwd_flag',
       u'trip_duration', u'start_time', u'month', u'day', u'hour', u'minute',
       u'second', u'weekday'],
      dtype='object')


In [33]:
# KNN regressor start_end_Lon_lat

min_max_scaler = MinMaxScaler()
train_x = train[["pickup_longitude","pickup_latitude","dropoff_longitude","dropoff_latitude"]].values
train_y = train["trip_duration"].values
test_x = test[["pickup_longitude","pickup_latitude","dropoff_longitude","dropoff_latitude"]].values
min_max_scaler.fit(np.concatenate([train_x,test_x]))
transform_train_x = min_max_scaler.transform(train_x)
transform_test_x = min_max_scaler.transform(test_x)
model = KNeighborsRegressor()
model.fit(transform_train_x,train_y)
predicts = model.predict(transform_test_x)
test["trip_duration"] = predicts
save_result(test[["id","trip_duration"]],"../result/start_end_Lon_Lat_KNN_default_avg.csv")

Index([u'id', u'vendor_id', u'pickup_datetime', u'dropoff_datetime',
       u'passenger_count', u'pickup_longitude', u'pickup_latitude',
       u'dropoff_longitude', u'dropoff_latitude', u'store_and_fwd_flag',
       u'trip_duration', u'start_time', u'month', u'day', u'hour', u'minute',
       u'second', u'weekday'],
      dtype='object')


In [44]:
# KNN regressor start_end_Lon_lat and hour,minute,second

min_max_scaler = MinMaxScaler()
train_x = train[["pickup_longitude","pickup_latitude","dropoff_longitude","dropoff_latitude"]].values
train_y = train["trip_duration"].values
test_x = test[["pickup_longitude","pickup_latitude","dropoff_longitude","dropoff_latitude"]].values
train_x_hour_minute_second = train[["hour","minute","second"]].values
train_x_seconds = train_x_hour_minute_second[:,0]*3600 + train_x_hour_minute_second[:,1]*60 + train_x_hour_minute_second[:,2]
train_x_seconds = train_x_seconds.reshape(len(train_x_seconds),1)
train_x = np.concatenate([train_x,train_x_seconds],axis=1)

test_x_hour_minute_second = test[["hour","minute","second"]].values
test_x_seconds = test_x_hour_minute_second[:,0]*3600 + test_x_hour_minute_second[:,1]*60 + test_x_hour_minute_second[:,2]
test_x_seconds = test_x_seconds.reshape(len(test_x_seconds),1)
test_x = np.concatenate([test_x,test_x_seconds],axis=1)

min_max_scaler.fit(np.concatenate([train_x,test_x]))
transform_train_x = min_max_scaler.transform(train_x)
transform_test_x = min_max_scaler.transform(test_x)
model = KNeighborsRegressor()
model.fit(transform_train_x,train_y)
predicts = model.predict(transform_test_x)
test["trip_duration"] = predicts
save_result(test[["id","trip_duration"]],"../result/start_end_Lon_Lat_hour_minute_second_KNN_default_avg.csv")

In [ ]:
# KNN regressor start_end_Lon_lat grid_search


params = {"n_neighbors":[10,20,30],"leaf_size":[30,50,70],"p":[1,2,3],"weights":["uniform","distance"]}
model = GridSearchCV(KNeighborsRegressor(),params,cv=5,verbose=2,scoring="neg_mean_squared_log_error")
print model

min_max_scaler = MinMaxScaler()
train_x = train[["pickup_longitude","pickup_latitude","dropoff_longitude","dropoff_latitude"]].values
train_y = train["trip_duration"].values
test_x = test[["pickup_longitude","pickup_latitude","dropoff_longitude","dropoff_latitude"]].values
min_max_scaler.fit(np.concatenate([train_x,test_x]))
transform_train_x = min_max_scaler.transform(train_x)
transform_test_x = min_max_scaler.transform(test_x)
# model = KNeighborsRegressor()
model.fit(transform_train_x,train_y)
predicts = model.predict(transform_test_x)
test["trip_duration"] = predicts
save_result(test[["id","trip_duration"]],"../result/start_end_Lon_Lat_KNN_gridsearch_avg.csv")

GridSearchCV(cv=5, error_score='raise',
       estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_neighbors': [10, 20, 30], 'weights': ['uniform', 'distance'], 'leaf_size': [30, 50, 70], 'p': [1, 2, 3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_squared_log_error', verbose=2)
Fitting 5 folds for each of 54 candidates, totalling 270 fits
[CV] n_neighbors=10, weights=uniform, leaf_size=30, p=1 ..............
[CV]  n_neighbors=10, weights=uniform, leaf_size=30, p=1, total=  16.5s
[CV] n_neighbors=10, weights=uniform, leaf_size=30, p=1 ..............


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.2min remaining:    0.0s


[CV]  n_neighbors=10, weights=uniform, leaf_size=30, p=1, total=  16.3s
[CV] n_neighbors=10, weights=uniform, leaf_size=30, p=1 ..............
[CV]  n_neighbors=10, weights=uniform, leaf_size=30, p=1, total=  15.2s
[CV] n_neighbors=10, weights=uniform, leaf_size=30, p=1 ..............
[CV]  n_neighbors=10, weights=uniform, leaf_size=30, p=1, total=  14.6s
[CV] n_neighbors=10, weights=uniform, leaf_size=30, p=1 ..............
[CV]  n_neighbors=10, weights=uniform, leaf_size=30, p=1, total=  14.5s
[CV] n_neighbors=10, weights=distance, leaf_size=30, p=1 .............
[CV]  n_neighbors=10, weights=distance, leaf_size=30, p=1, total=  15.3s
[CV] n_neighbors=10, weights=distance, leaf_size=30, p=1 .............
[CV]  n_neighbors=10, weights=distance, leaf_size=30, p=1, total=  14.6s
[CV] n_neighbors=10, weights=distance, leaf_size=30, p=1 .............
[CV]  n_neighbors=10, weights=distance, leaf_size=30, p=1, total=  15.4s
[CV] n_neighbors=10, weights=distance, leaf_size=30, p=1 ..........